In [1]:
import os, sys, getopt 
os.environ['KERAS_BACKEND'] = 'theano'
import yaml
import cPickle
import numpy as np
import keras
from matplotlib import pyplot as plt 
import keras.callbacks
from keras.datasets import mnist
#from keras.utils.visualize_util import plot
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, RMSprop
from model import build_model, MyCallback
from keras.callbacks import CSVLogger
from data import load
from sklearn.model_selection import train_test_split
import keras.backend as K
import theano.tensor as T
from theano import function
#np.set_printoptions(threshold=10000000)
from theano.printing import return_value,reset_value
keras.__version__

Using Theano backend.


'1.2.1'

In [2]:
%run model_new.ipynb
dataset = '990622'
old = []
open('/home/rakesh/Desktop/zCap/new/log_hbp19_0.acc','w').close()  # clear previous data
old_len = 0
second_epoch = 15

Model Loaded


In [3]:

def build_data_dict(in_name, out_name, in_data, out_data):
    in_dict = dict()
    in_dict[in_name] = in_data
    
    out_dict = dict((k, out_data) for k in out_name)
    return (in_dict, out_dict)

def build_loss_weight(config):
    if config['hedge'] == False:
        w = [1.]
    elif config['loss_weight'] == 'ave':
        w = [1./ config['n_layers']]* config['n_layers']
    return w
def eer(y_true, y_pred):
    y_true.eval()
    #print type(K.mean(y_pred))
    x1 = T.dscalar('x')
    y1 = T.dscalar('x')
    f = function([x], z)
    
    return K.mean(y_pred)

def binary_accuracy(y_true, y_pred):
    
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    if print_mode==1:
        y_pred = K.print_tensor(y_pred)
    #K.print_tensor(y_pred)
    #K.get_value(y_pred)
    return K.mean(K.equal(y_true, K.round(y_pred)))

In [4]:
def main(arg, idx=0):
    config = {'learning_rate': 1e-3,
              'optim': 'Adam',
              'batch_size': 1,
              'nb_epoch': 50,
              'n_layers': 10,
              'hidden_num': 100,
              'activation': 'relu',
              'loss_weight': 'ave',
              'adaptive_weight': False,
              'data': 'mnist',
              'hedge': True,
              'log': 'mnist_hedge.log'}

    configfile = ''
    opts = [('-c', 'hb19.yaml')]
    args = []
    for opt, arg in opts:
        if opt == '-h':
            print (helpstring)
            yamlstring = yaml.dump(config,default_flow_style=False,explicit_start=True)
            print("YAML configuration file format:")
            print("")
            print("%YAML 1.2")
            print(yamlstring)
            sys.exit()

        elif opt in ('-c', '--config'):
            configfile = arg

        print("Config file is %s" % configfile)

    if os.path.exists(configfile):
        f = open(configfile)
        user_config = yaml.load(f.read())
        config.update(user_config)
    
    print("Printing configuration:")
    for key,value in config.iteritems():
        print("  ",key,": ",value)
        
    (X_train, Y_train, X_test, Y_test, nb_classes) = load(dataset)
    (_, _, x_test, y_test, nb_classes) = load(dataset)
    
    print np.array_equal(Y_test,y_test)
    
    Y_TRAIN = Y_train.reshape(-1)
    Y_TEST = Y_test.reshape(-1)
    
    train_len = Y_TRAIN.shape[0]
    test_len = Y_TEST.shape[0]
    
    Y_train = np.zeros((train_len,2))
    Y_test = np.zeros((test_len,2))
    
    Y_train[np.arange(train_len), Y_TRAIN] = 1
    Y_test[np.arange(test_len), Y_TEST] = 1
    
    
    #(X_test, Y_test, X_train, Y_train, nb_classes) = load(config['data'])
    #print Y_test
    #X_train = X_train[:500000]
    #Y_train = Y_train[:500000]
    '''
    X_test = X_test[:1]
    Y_test = Y_test[:1]'''
    '''
    # data for concept drift data 
    import numpy as np

    X = np.load('/home/rakesh/Desktop/zCap/x_train.npy')
    y = np.load('/home/rakesh/Desktop/zCap/y_train.npy')
    
    X_test, X_train, Y_test, Y_train = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.211)
    '''
    model, in_name, out_name = build_model(config)
    if len(out_name) == 1:
        out_name_loss = ['loss']
    else:
        out_name_loss = [s + '_loss' for s in out_name]

    model.summary()
    
    #print Y_test[:,1].sum()
    
    #plot(model, to_file = 'model.png')
    
    optim = eval(config['optim'])(lr = config['learning_rate'])
    in_dict, out_dict = build_data_dict(in_name, out_name, X_train, Y_train)
    in_val, out_val = build_data_dict(in_name, out_name, X_test, Y_test)
    loss_dict = dict((k, 'categorical_crossentropy') for k in out_name) 
  
    loss_weights = build_loss_weight(config)
    my_callback = MyCallback( loss_weights, names = out_name_loss, hedge = config['hedge'], log_name = config['log'])
    #csv  = CSVLogger(config['log'])
    
    
    model.compile(optimizer = optim, loss = loss_dict, hedge = config['hedge'],loss_weights = loss_weights, metrics =['accuracy'])#,binary_accuracy]) #['accuracy'])
    
    history = model.fit(in_dict, out_dict,validation_split=0.05, nb_epoch = config['nb_epoch'], batch_size = config['batch_size'], callbacks=[my_callback] , verbose = 1)
    
    # ------------------- save details of first training --------------
    cumLoss = np.cumsum(my_callback.acc)
    indexOfLoss = np.arange(len(cumLoss))+1
    cumAverageLoss = cumLoss/indexOfLoss
    filename = (config['log'] + '_' + str(idx) + '.acc')
    np.savetxt(filename, cumAverageLoss, delimiter=',')
    
    #--------------------------------------------------------------
    hbp = open('/home/rakesh/Desktop/zCap/new/log_hbp19_0.acc','r')
    
    for line in hbp.readlines():
        old.append(float(line))
    old_len = len(old)
    #open('/home/rakesh/Desktop/zCap/new/log_hbp19_0.acc','w').close()    
    history = model.fit(in_val, out_val,
                        nb_epoch = second_epoch+config['nb_epoch'],
                        
                        batch_size = config['batch_size'], callbacks=[my_callback] , 
                        verbose = 1,
                        
                        initial_epoch =config['nb_epoch'])
    #print history.keys()
    #print model.evaluate(X_test,Y_test)
    #print my_callback.acc
    
    k=return_value()
    #print(k[0].shape)
    
    #----------------------   save details of second training  ---------------------
    cumLoss = np.cumsum(my_callback.acc)
    indexOfLoss = np.arange(len(cumLoss))+1
    cumAverageLoss = cumLoss/indexOfLoss
    filename = (config['log'] + '_' + str(idx) + '.acc')
    np.savetxt(filename, cumAverageLoss, delimiter=',')
    
    return my_callback,history,model,x_test,y_test,in_val,out_val,k

In [5]:
history = None
Model1 = None
X_test = None
Y_test = None
in_val = None
out_val = None
my_callback = None
k=None
if __name__ == '__main__':
    #for i in range(5):
    my_callback,history,Model1,X_test,Y_test,in_val,out_val,k = main(sys.argv[1:], 0)

Config file is hb19.yaml
Printing configuration:
('  ', 'optim', ': ', 'SGD')
('  ', 'activation', ': ', 'relu')
('  ', 'n_layers', ': ', 10)
('  ', 'batch_size', ': ', 50)
('  ', 'hidden_num', ': ', 100)
('  ', 'hedge', ': ', True)
('  ', 'data', ': ', 'all_21')
('  ', 'input_size', ': ', 21)
('  ', 'log', ': ', 'log_hbp19')
('  ', 'nb_epoch', ': ', 45)
('  ', 'adaptive_weight', ': ', True)
('  ', 'model', ': ', 'MLP')
('  ', 'loss_weight', ': ', 'ave')
('  ', 'learning_rate', ': ', 0.005)
True
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
in0 (InputLayer)                 (None, 21)            0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           2200        in0[0][0]                   

314735/314735 [==============================] - 41s - loss: 0.7223 - out0_loss: 5.9407 - out1_loss: 2.1556 - out2_loss: 1.0283 - out3_loss: 0.7404 - out4_loss: 0.6483 - out5_loss: 0.6290 - out6_loss: 0.6138 - out7_loss: 0.5989 - out8_loss: 0.5922 - out9_loss: 0.5881 - out0_acc: 0.6114 - out1_acc: 0.6151 - out2_acc: 0.6296 - out3_acc: 0.6510 - out4_acc: 0.6636 - out5_acc: 0.6626 - out6_acc: 0.6649 - out7_acc: 0.6659 - out8_acc: 0.6658 - out9_acc: 0.6692 - weighted_acc: 0.6680 - val_loss: 0.6715 - val_out0_loss: 4.8442 - val_out1_loss: 1.0906 - val_out2_loss: 0.7924 - val_out3_loss: 0.6480 - val_out4_loss: 0.6559 - val_out5_loss: 0.6356 - val_out6_loss: 0.6359 - val_out7_loss: 0.6056 - val_out8_loss: 0.6128 - val_out9_loss: 0.6040 - val_out0_acc: 0.5918 - val_out1_acc: 0.6093 - val_out2_acc: 0.6555 - val_out3_acc: 0.6008 - val_out4_acc: 0.5999 - val_out5_acc: 0.6370 - val_out6_acc: 0.6309 - val_out7_acc: 0.6484 - val_out8_acc: 0.6415 - val_out9_acc: 0.6465 - val_weighted_acc: 0.6375.585

314735/314735 [==============================] - 32s - loss: 0.4846 - out0_loss: 0.8260 - out1_loss: 0.5931 - out2_loss: 0.5929 - out3_loss: 0.5746 - out4_loss: 0.5607 - out5_loss: 0.5555 - out6_loss: 0.5490 - out7_loss: 0.5188 - out8_loss: 0.5125 - out9_loss: 0.4744 - out0_acc: 0.6306 - out1_acc: 0.6711 - out2_acc: 0.6719 - out3_acc: 0.6735 - out4_acc: 0.6829 - out5_acc: 0.6738 - out6_acc: 0.6828 - out7_acc: 0.7161 - out8_acc: 0.7254 - out9_acc: 0.7552 - weighted_acc: 0.7553 - val_loss: 0.4747 - val_out0_loss: 0.8855 - val_out1_loss: 0.6492 - val_out2_loss: 0.5914 - val_out3_loss: 0.5756 - val_out4_loss: 0.5698 - val_out5_loss: 0.5674 - val_out6_loss: 0.5655 - val_out7_loss: 0.5235 - val_out8_loss: 0.5094 - val_out9_loss: 0.4619 - val_out0_acc: 0.6630 - val_out1_acc: 0.6456 - val_out2_acc: 0.6846 - val_out3_acc: 0.6730 - val_out4_acc: 0.6774 - val_out5_acc: 0.6577 - val_out6_acc: 0.6871 - val_out7_acc: 0.7166 - val_out8_acc: 0.7424 - val_out9_acc: 0.7808 - val_weighted_acc: 0.7650out1

314735/314735 [==============================] - 36s - loss: 0.4395 - out0_loss: 0.9732 - out1_loss: 0.5745 - out2_loss: 0.5755 - out3_loss: 0.5649 - out4_loss: 0.5589 - out5_loss: 0.5530 - out6_loss: 0.5462 - out7_loss: 0.5093 - out8_loss: 0.4946 - out9_loss: 0.4242 - out0_acc: 0.6256 - out1_acc: 0.6820 - out2_acc: 0.6853 - out3_acc: 0.6847 - out4_acc: 0.6872 - out5_acc: 0.6786 - out6_acc: 0.6853 - out7_acc: 0.7262 - out8_acc: 0.7396 - out9_acc: 0.7882 - weighted_acc: 0.7875 - val_loss: 0.5065 - val_out0_loss: 0.7381 - val_out1_loss: 0.6009 - val_out2_loss: 0.6186 - val_out3_loss: 0.5981 - val_out4_loss: 0.5745 - val_out5_loss: 0.5672 - val_out6_loss: 0.5549 - val_out7_loss: 0.5374 - val_out8_loss: 0.5268 - val_out9_loss: 0.4982 - val_out0_acc: 0.6507 - val_out1_acc: 0.6692 - val_out2_acc: 0.6740 - val_out3_acc: 0.6255 - val_out4_acc: 0.6948 - val_out5_acc: 0.6869 - val_out6_acc: 0.6876 - val_out7_acc: 0.7156 - val_out8_acc: 0.7219 - val_out9_acc: 0.7743 - val_weighted_acc: 0.7705out4

314735/314735 [==============================] - 33s - loss: 0.4315 - out0_loss: 1.0064 - out1_loss: 0.5689 - out2_loss: 0.5732 - out3_loss: 0.5631 - out4_loss: 0.5613 - out5_loss: 0.5539 - out6_loss: 0.5462 - out7_loss: 0.5085 - out8_loss: 0.4916 - out9_loss: 0.4151 - out0_acc: 0.6270 - out1_acc: 0.6880 - out2_acc: 0.6865 - out3_acc: 0.6876 - out4_acc: 0.6826 - out5_acc: 0.6774 - out6_acc: 0.6833 - out7_acc: 0.7234 - out8_acc: 0.7399 - out9_acc: 0.7946 - weighted_acc: 0.7942 - val_loss: 0.4169 - val_out0_loss: 0.7897 - val_out1_loss: 0.5474 - val_out2_loss: 0.5435 - val_out3_loss: 0.5493 - val_out4_loss: 0.5708 - val_out5_loss: 0.5566 - val_out6_loss: 0.5433 - val_out7_loss: 0.5115 - val_out8_loss: 0.4893 - val_out9_loss: 0.4020 - val_out0_acc: 0.6365 - val_out1_acc: 0.7071 - val_out2_acc: 0.6972 - val_out3_acc: 0.6973 - val_out4_acc: 0.6805 - val_out5_acc: 0.6724 - val_out6_acc: 0.6786 - val_out7_acc: 0.6901 - val_out8_acc: 0.7251 - val_out9_acc: 0.7905 - val_weighted_acc: 0.7927 out

...

314735/314735 [==============================] - 32s - loss: 0.3465 - out0_loss: 1.2567 - out1_loss: 0.5353 - out2_loss: 0.5125 - out3_loss: 0.5076 - out4_loss: 0.5072 - out5_loss: 0.5011 - out6_loss: 0.4961 - out7_loss: 0.4467 - out8_loss: 0.4132 - out9_loss: 0.3239 - out0_acc: 0.6479 - out1_acc: 0.7178 - out2_acc: 0.7317 - out3_acc: 0.7288 - out4_acc: 0.7305 - out5_acc: 0.7368 - out6_acc: 0.7406 - out7_acc: 0.7845 - out8_acc: 0.8059 - out9_acc: 0.8502 - weighted_acc: 0.8497 - val_loss: 0.3508 - val_out0_loss: 0.7662 - val_out1_loss: 0.5658 - val_out2_loss: 0.5245 - val_out3_loss: 0.5415 - val_out4_loss: 0.5316 - val_out5_loss: 0.5238 - val_out6_loss: 0.5117 - val_out7_loss: 0.4585 - val_out8_loss: 0.4236 - val_out9_loss: 0.3322 - val_out0_acc: 0.6706 - val_out1_acc: 0.6840 - val_out2_acc: 0.7014 - val_out3_acc: 0.6871 - val_out4_acc: 0.6976 - val_out5_acc: 0.6923 - val_out6_acc: 0.7014 - val_out7_acc: 0.7908 - val_out8_acc: 0.8202 - val_out9_acc: 0.8555 - val_weighted_acc: 0.8569out1

314735/314735 [==============================] - 27s - loss: 0.3329 - out0_loss: 1.2798 - out1_loss: 0.5273 - out2_loss: 0.5031 - out3_loss: 0.4918 - out4_loss: 0.4887 - out5_loss: 0.4856 - out6_loss: 0.4777 - out7_loss: 0.4301 - out8_loss: 0.3922 - out9_loss: 0.3100 - out0_acc: 0.6505 - out1_acc: 0.7264 - out2_acc: 0.7382 - out3_acc: 0.7436 - out4_acc: 0.7470 - out5_acc: 0.7500 - out6_acc: 0.7599 - out7_acc: 0.7956 - out8_acc: 0.8195 - out9_acc: 0.8573 - weighted_acc: 0.8569 - val_loss: 0.3378 - val_out0_loss: 2.5800 - val_out1_loss: 0.5717 - val_out2_loss: 0.5200 - val_out3_loss: 0.4987 - val_out4_loss: 0.4777 - val_out5_loss: 0.4813 - val_out6_loss: 0.4690 - val_out7_loss: 0.4261 - val_out8_loss: 0.3930 - val_out9_loss: 0.3006 - val_out0_acc: 0.5934 - val_out1_acc: 0.6598 - val_out2_acc: 0.7104 - val_out3_acc: 0.7126 - val_out4_acc: 0.7433 - val_out5_acc: 0.7527 - val_out6_acc: 0.7748 - val_out7_acc: 0.7973 - val_out8_acc: 0.8322 - val_out9_acc: 0.8698 - val_weighted_acc: 0.8722
Epo

314735/314735 [==============================] - 28s - loss: 0.3262 - out0_loss: 1.2764 - out1_loss: 0.5204 - out2_loss: 0.4928 - out3_loss: 0.4798 - out4_loss: 0.4781 - out5_loss: 0.4759 - out6_loss: 0.4677 - out7_loss: 0.4170 - out8_loss: 0.3803 - out9_loss: 0.3037 - out0_acc: 0.6544 - out1_acc: 0.7320 - out2_acc: 0.7452 - out3_acc: 0.7536 - out4_acc: 0.7533 - out5_acc: 0.7538 - out6_acc: 0.7694 - out7_acc: 0.8019 - out8_acc: 0.8237 - out9_acc: 0.8613 - weighted_acc: 0.8609 - val_loss: 0.2868 - val_out0_loss: 1.1719 - val_out1_loss: 0.4718 - val_out2_loss: 0.4548 - val_out3_loss: 0.4542 - val_out4_loss: 0.4561 - val_out5_loss: 0.4649 - val_out6_loss: 0.4499 - val_out7_loss: 0.3953 - val_out8_loss: 0.3535 - val_out9_loss: 0.2638 - val_out0_acc: 0.6756 - val_out1_acc: 0.7704 - val_out2_acc: 0.7964 - val_out3_acc: 0.7678 - val_out4_acc: 0.7759 - val_out5_acc: 0.7410 - val_out6_acc: 0.7726 - val_out7_acc: 0.8278 - val_out8_acc: 0.8526 - val_out9_acc: 0.8963 - val_weighted_acc: 0.8953
Epo

58465/58465 [==============================] - 5s - loss: 0.3221 - out0_loss: 1.2698 - out1_loss: 0.5133 - out2_loss: 0.4880 - out3_loss: 0.4749 - out4_loss: 0.4710 - out5_loss: 0.4722 - out6_loss: 0.4622 - out7_loss: 0.4153 - out8_loss: 0.3713 - out9_loss: 0.2997 - out0_acc: 0.6543 - out1_acc: 0.7383 - out2_acc: 0.7497 - out3_acc: 0.7569 - out4_acc: 0.7600 - out5_acc: 0.7626 - out6_acc: 0.7726 - out7_acc: 0.7976 - out8_acc: 0.8273 - out9_acc: 0.8614 - weighted_acc: 0.8616      ETA: 4s - loss: 0.3226 - out0_loss: 1.2740 - out1_loss: 0.5201 - out2_loss: 0.4900 - out3_loss: 0.4764 - out4_loss: 0.4719 - out5_loss: 0.4737 - out6_loss: 0.4634 - out7_loss: 0.4159 - out8_loss: 0.3705 - out9_loss: 0.3000 - out0_acc: 0.6463 - out1_acc: 0.7325 - out2_acc: 0.7472 - out3_acc: 0.7562 - out4_acc: 0.7628 - out5_acc: 0.7595 - out6_acc: 0.7705 - out7_acc: 0.7958 - out8_acc: 0. - ETA: 4s - loss: 0.3248 - out0_loss: 1.2719 -  - ETA: 1s - loss: 0.3227 - out0_loss: 1.3074 - out1_loss: 0.5149 - out2_loss: 0

58465/58465 [==============================] - 5s - loss: 0.3184 - out0_loss: 1.2864 - out1_loss: 0.5192 - out2_loss: 0.4862 - out3_loss: 0.4735 - out4_loss: 0.4688 - out5_loss: 0.4692 - out6_loss: 0.4553 - out7_loss: 0.4058 - out8_loss: 0.3650 - out9_loss: 0.2957 - out0_acc: 0.6584 - out1_acc: 0.7389 - out2_acc: 0.7523 - out3_acc: 0.7602 - out4_acc: 0.7610 - out5_acc: 0.7664 - out6_acc: 0.7770 - out7_acc: 0.8071 - out8_acc: 0.8328 - out9_acc: 0.8656 - weighted_acc: 0.8652      ETA: 1s - loss: 0.3159 - out0_loss: 1.2644 - out1_loss: 0.5124 - out2_loss: 0.4835 - out3_loss: 0.4713 - out4_loss: 0.4659 - out5_loss: 0.4650 - out6_loss: 0.4527 - out7_loss: 0.4032 
Epoch 55/60
58465/58465 [==============================] - 5s - loss: 0.3148 - out0_loss: 1.2582 - out1_loss: 0.5156 - out2_loss: 0.4785 - out3_loss: 0.4658 - out4_loss: 0.4619 - out5_loss: 0.4637 - out6_loss: 0.4512 - out7_loss: 0.4019 - out8_loss: 0.3602 - out9_loss: 0.2926 - out0_acc: 0.6626 - out1_acc: 0.7401 - out2_acc: 0.7559

In [ ]:
reset_value()
print return_value()
metrics = Model1.evaluate(in_val,out_val,verbose=1)
k=return_value()

In [ ]:
metrics

In [ ]:
y_pred = np.concatenate(k)
y_true = out_val[out_val.keys()[-1]]

## ROC

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc

In [ ]:
y_scores = y_pred[:,1]
y_true = y_true[:,1]
y_true.shape

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_scores, pos_label=1)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]

In [ ]:
print "FPR",fpr.shape
print "TPR",tpr.shape
print "thresholds",thresholds

In [ ]:
print "AUC",roc_auc_score(y_true, y_scores)
print "EER",EER

In [ ]:
fnr[np.nanargmin(np.absolute((fnr - fpr)))]

## ROC Curve

In [ ]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d
eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
thresh = interp1d(fpr, thresholds)(eer)
print "EER",eer

In [ ]:
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_true, y_scores))

plt.plot(fpr, tpr, 'b', label = 'EER = %0.2f' % eer)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(dataset+' ROC - 50ms ')
plt.legend(loc="lower right")
plt.savefig(dataset+'_plot_hbp_roc.png',dpi=100)

plt.clf() 

## Error rate while whole training

In [6]:
hbp = open('/home/rakesh/Desktop/zCap/new/log_hbp19_0.acc','r')

print "old len : ",len(old)

xhbp = []

for line in hbp.readlines():
    xhbp.append(float(line))
print "new len : ",len(xhbp)    
#train_acc,_= get_metrics()    
    
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)


plt.plot(1-np.array(xhbp), label = 'online',color = 'r',linestyle='--')
plt.plot(1-np.array(old), label = 'initial train',color = 'b')
plt.legend()
plt.title(dataset+' 50ms 21 features HBP')
plt.ylabel('Error Rate')
#plt.show()
plt.savefig(dataset+'_plot_online_45_15_hbp.png',dpi=100)

plt.clf()  

old len :  283275
new len :  300825


<Figure size 576x432 with 0 Axes>

In [ ]:
model = Model1
Model1.fit(in_val, out_val, nb_epoch = 1, batch_size = 1, callbacks=[my_callback] , verbose = 1)

In [ ]:
hbp = open('/home/rakesh/Desktop/zCap/new/log_hbp19_0.acc','r')

x1hbp = []

for line in hbp.readlines():
    x1hbp.append(float(line))
    
    
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)

plt.plot(1-np.array(xhbp), label = 'online',color = 'r')
plt.plot(1-np.array(old), label = 'train',color = 'b')
plt.legend()
plt.title(dataset+' 50ms 21 features HBP')
plt.ylabel('Error Rate')
#plt.show()
plt.savefig(dataset+'_plot_hbp_cd.png',dpi=100)

plt.clf()  

In [ ]:
##v  train and val curve
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)

plt.plot(np.array(history.history['weighted_acc']), label = 'Training')
plt.plot(np.array(history.history['val_weighted_acc']), label = 'Validation')
plt.legend()
plt.title(dataset+' | 50ms | Test_acc : %0.3f'%metrics[-1])
plt.ylabel('Accuracy')
#plt.show()
plt.savefig(dataset+'_plot_train_val.png',dpi=100)

plt.clf()